In [1]:
import nltk
import numpy as np
from sklearn.model_selection import train_test_split

# 下载数据

In [ ]:
nltk.download('treebank')

In [2]:
tagged_sentences = nltk.corpus.treebank.tagged_sents()
 
print(tagged_sentences[0])
print("Tagged sentences: ", len(tagged_sentences))
print("Tagged words:", len(nltk.corpus.treebank.tagged_words()))

[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]
Tagged sentences:  3914
Tagged words: 100676


# Preprocess

In [3]:
import numpy as np

sentences, sentence_tags =[], [] 
for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append(np.array(sentence))
    sentence_tags.append(np.array(tags))
print(sentences[5])
print(sentence_tags[5])

['Lorillard' 'Inc.' ',' 'the' 'unit' 'of' 'New' 'York-based' 'Loews'
 'Corp.' 'that' '*T*-2' 'makes' 'Kent' 'cigarettes' ',' 'stopped' 'using'
 'crocidolite' 'in' 'its' 'Micronite' 'cigarette' 'filters' 'in' '1956'
 '.']
['NNP' 'NNP' ',' 'DT' 'NN' 'IN' 'JJ' 'JJ' 'NNP' 'NNP' 'WDT' '-NONE-' 'VBZ'
 'NNP' 'NNS' ',' 'VBD' 'VBG' 'NN' 'IN' 'PRP$' 'NN' 'NN' 'NNS' 'IN' 'CD'
 '.']


In [4]:
from sklearn.model_selection import train_test_split
(train_sentences, 
 test_sentences, 
 train_tags, 
 test_tags) = train_test_split(sentences, sentence_tags, test_size=0.2)

In [5]:
words, tags = set([]), set([])
 
for s in train_sentences:
    for w in s:
        words.add(w.lower())

for ts in train_tags:
    for t in ts:
        tags.add(t)

print(len(words),len(tags))

10047 46


### 建立 词-->index, tag-->index 的索引

In [6]:
word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs
 
tag2index = {t: i + 1 for i, t in enumerate(list(tags))}
tag2index['-PAD-'] = 0  # The special value used to padding

### 数字化表示

In [7]:
train_sentences_X, test_sentences_X, train_tags_y, test_tags_y = [], [], [], []
 
for s in train_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sentences_X.append(s_int)

for s in test_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sentences_X.append(s_int)

for s in train_tags:
    train_tags_y.append([tag2index[t] for t in s])

for s in test_tags:
    test_tags_y.append([tag2index[t] for t in s])

print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[6983, 1692, 1117, 806, 317, 4492, 6390, 4783, 260, 1719, 2961, 1585, 8222, 4906, 6899, 3478, 6243]
[4578, 9492, 1968, 4573, 7340, 6649, 823, 4843, 4737, 4950, 5327, 4139, 8222, 926, 8987, 7340, 6400, 9202, 8453, 6992, 6243]
[36, 5, 6, 37, 18, 36, 19, 35, 33, 29, 29, 5, 18, 29, 29, 5, 14]
[11, 36, 19, 18, 15, 28, 13, 3, 30, 28, 18, 19, 18, 29, 19, 15, 42, 8, 30, 2, 14]


In [8]:
MAX_LENGTH = len(max(train_sentences_X, key=len))
print(MAX_LENGTH)  # 271

128


### 用 keras 做 padding

In [9]:
from keras.preprocessing.sequence import pad_sequences
 
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')
 
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

Using TensorFlow backend.


[6983 1692 1117  806  317 4492 6390 4783  260 1719 2961 1585 8222 4906
 6899 3478 6243    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[4578 9492 1968 4573 7340 6649  823 4843 4737 4950 5327 4139 8222  926
 8987 7340 6400 9202 8453 6992 6243    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0

In [10]:
print(len(train_sentences_X[0]))
print(len(test_sentences_X[0]))
print(len(train_tags_y[0]))
print(len(test_tags_y[0]))

128
128
128
128


### 建立 architecture 

from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam
import tensorflow as tf
 
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])

model.summary()

In [12]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [13]:
cat_train_tags_y = to_categorical(train_tags_y, len(tag2index))
print(cat_train_tags_y[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [14]:
print(len(cat_train_tags_y),len(cat_train_tags_y[0]))

3131 128


### Tensorboard

In [15]:
from tensorflow.keras.callbacks import TensorBoard
import time

tensorboard = TensorBoard(log_dir="logs/ner_{}".format(int(time.time())))

In [16]:
len(train_sentences_X)

3131

### 运行模型，注意下初始化，否则有时会报错

In [19]:
import keras

keras.backend.get_session().run(tf.global_variables_initializer()) ##初始化

model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), 
          batch_size=128, epochs=20, validation_split=0.2,callbacks=[tensorboard])

Train on 2504 samples, validate on 627 samples
Epoch 1/20
2504/2504 [==============================] - 15s 6ms/step - loss: 1.6482 - acc: 0.7603 - val_loss: 0.8111 - val_acc: 0.7978
Epoch 2/20
2504/2504 [==============================] - 15s 6ms/step - loss: 0.7035 - acc: 0.8028 - val_loss: 0.6828 - val_acc: 0.8043
Epoch 3/20
2504/2504 [==============================] - 15s 6ms/step - loss: 0.6444 - acc: 0.8215 - val_loss: 0.6488 - val_acc: 0.8208
Epoch 4/20
2504/2504 [==============================] - 15s 6ms/step - loss: 0.6147 - acc: 0.8262 - val_loss: 0.6209 - val_acc: 0.8205
Epoch 5/20
2504/2504 [==============================] - 15s 6ms/step - loss: 0.6002 - acc: 0.8253 - val_loss: 0.6144 - val_acc: 0.8244
Epoch 6/20
2504/2504 [==============================] - 15s 6ms/step - loss: 0.5864 - acc: 0.8277 - val_loss: 0.5997 - val_acc: 0.8267
Epoch 7/20
2504/2504 [==============================] - 15s 6ms/step - loss: 0.5740 - acc: 0.8340 - val_loss: 0.5892 - val_acc: 0.8350
Epoch 8/

### 模型评估

In [20]:
scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))
print(f"{model.metrics_names[1]}: {scores[1] * 100}")   # acc: 99.09751977804825

783/783 [==============================] - 3s 3ms/step
acc: 97.88673370886275


### 个例预测

In [21]:
test_samples = [
    "running is very important for me .".split(),
    "I was running every day for a month .".split()
]
print(test_samples)

[['running', 'is', 'very', 'important', 'for', 'me', '.'], ['I', 'was', 'running', 'every', 'day', 'for', 'a', 'month', '.']]


In [22]:
test_samples_X = []
for s in test_samples:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_samples_X.append(s_int)

test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
print(test_samples_X)

[[5730 4783 3631 8391 8222  131 6243    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [9566 5340 5730 3718 5283 8222 9492 7242 6243    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0

In [23]:
predictions = model.predict(test_samples_X)
print(predictions, predictions.shape)

[[[1.41806458e-03 2.09526694e-03 1.12760561e-02 ... 3.17072519e-03
   2.91167671e-04 3.01697964e-05]
  [3.85120438e-06 1.51414424e-04 8.21110909e-04 ... 9.93145513e-04
   3.02747412e-05 4.28981526e-04]
  [2.96232629e-05 4.70056169e-04 5.68074989e-04 ... 2.62343604e-03
   6.95341587e-05 1.43037367e-04]
  ...
  [9.99962807e-01 7.65429220e-10 1.07162827e-10 ... 6.33166436e-11
   2.01757988e-09 6.23867752e-14]
  [9.99942541e-01 1.76822768e-09 1.55685145e-10 ... 1.27181918e-10
   4.58827953e-09 1.68707428e-13]
  [9.99896049e-01 4.06278611e-09 2.48196269e-10 ... 2.64020694e-10
   1.04044124e-08 4.84820029e-13]]

 [[6.17544913e-07 1.01221405e-04 8.78206105e-04 ... 3.96122487e-04
   1.96385026e-05 8.66525690e-04]
  [3.16309581e-07 4.27320410e-05 3.45051847e-02 ... 4.56528884e-04
   1.12630059e-05 2.42128968e-03]
  [1.18084135e-04 1.23343151e-03 3.70652117e-02 ... 2.70214025e-03
   1.19684650e-04 2.45355841e-05]
  ...
  [9.99962807e-01 7.65513875e-10 1.07058223e-10 ... 6.32958755e-11
   2.01774

In [24]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences

In [25]:
#  "running is very important for me .".split(),
#   "I was running every day for a month .".split()
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))

[['NNP', 'VBZ', 'RB', 'JJ', 'IN', 'PRP', '.', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', 

## ignore padding 部分的占位符

In [26]:
from keras import backend as K
 
def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
 
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy

### 重新训练

In [27]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam
 
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy', ignore_class_accuracy(0)])
 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 128, 128)          1286272   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128, 512)          788480    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 128, 47)           24111     
_________________________________________________________________
activation_2 (Activation)    (None, 128, 47)           0         
Total params: 2,098,863
Trainable params: 2,098,863
Non-trainable params: 0
_________________________________________________________________


In [29]:
tensorboard = TensorBoard(log_dir="logs/ignore_ner_{}".format(int(time.time())))

keras.backend.get_session().run(tf.global_variables_initializer())

model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), 
          batch_size=128, epochs=50, validation_split=0.2, callbacks=[tensorboard])

Train on 2504 samples, validate on 627 samples
Epoch 1/50
2504/2504 [==============================] - 15s 6ms/step - loss: 1.6804 - acc: 0.7606 - ignore_accuracy: 0.0294 - val_loss: 0.8282 - val_acc: 0.7956 - val_ignore_accuracy: 0.1531
Epoch 2/50
2504/2504 [==============================] - 15s 6ms/step - loss: 0.7237 - acc: 0.8008 - ignore_accuracy: 0.0064 - val_loss: 0.6918 - val_acc: 0.7955 - val_ignore_accuracy: 0.0981
Epoch 3/50
2504/2504 [==============================] - 15s 6ms/step - loss: 0.6534 - acc: 0.8205 - ignore_accuracy: 0.1261 - val_loss: 0.6533 - val_acc: 0.8204 - val_ignore_accuracy: 0.1286
Epoch 4/50
2504/2504 [==============================] - 15s 6ms/step - loss: 0.6214 - acc: 0.8266 - ignore_accuracy: 0.1360 - val_loss: 0.6258 - val_acc: 0.8206 - val_ignore_accuracy: 0.1293
Epoch 5/50
2504/2504 [==============================] - 15s 6ms/step - loss: 0.5968 - acc: 0.8265 - ignore_accuracy: 0.1365 - val_loss: 0.6094 - val_acc: 0.8251 - val_ignore_accuracy: 0.148

In [30]:
# [['running', 'is', 'very', 'important', 'for', 'me', '.'],
#  ['I', 'was', 'running', 'every', 'day', 'for', 'a', 'month', '.']]


predictions = model.predict(test_samples_X)
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))

[['VBG', 'VBZ', 'RB', 'JJ', 'IN', 'PRP', '.', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', 